In [ ]:
!pip install pandas geopandas pyarrow shapely

In [ ]:
import pandas as pd
import geopandas
from shapely.geometry import Point
import os
from datetime import datetime

# Google Drive'ı bağlamak için (Colab'da çalıştırılıyorsa)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Google Drive'daki dosya yollarını belirtin
# LÜTFEN BU YOLLARI KENDİ GOOGLE DRIVE YAPINIZA GÖRE GÜNCELLEYİN!
base_drive_path = '/content/drive/MyDrive/' # Genellikle '/content/drive/MyDrive/' olur

mobility_data_filename = 'MobilityDataMay2024.parquet'
polygons_folder_name = 'Polygons/'

mobility_data_path = os.path.join(base_drive_path, mobility_data_filename)
polygons_path = os.path.join(base_drive_path, polygons_folder_name)

# Mobilite Verisi Yükleme
try:
    df_mobility = pd.read_parquet(mobility_data_path)
    print(f"Mobilite verisi '{mobility_data_path}' başarıyla yüklendi. Boyut: {df_mobility.shape}")
    print("İlk 5 satır:")
    print(df_mobility.head())
    print("\nVeri tipleri:")
    print(df_mobility.dtypes.head())
except FileNotFoundError:
    print(f"HATA: Mobilite verisi dosyası bulunamadı: {mobility_data_path}")
    print("Lütfen dosya yolunu ve adını kontrol edin.")
except Exception as e:
    print(f"HATA: Mobilite verisi yüklenirken bir sorun oluştu: {e}")

Mobilite verisi '/content/drive/MyDrive/MobilityDataMay2024.parquet' başarıyla yüklendi. Boyut: (52225739, 19)
İlk 5 satır:
    timestamp                            device_aid   latitude  longitude  \
0  1716563371  4d4b939f-3f71-4366-aea0-ebfbb1906b9f  40.987983  29.027896   
1  1716564390  30c55c28-45e8-6980-ad1d-7064b7c84051  40.988467  29.109296   
2  1716563418  d9287169-2268-4958-89dd-192d25dec817  41.023274  29.094867   
3  1716562879  e55c0589-0589-4539-9783-f96448f259f2  40.994214  29.151767   
4  1716563925  35b845d0-fba5-412a-a8f1-f987fa94ffaa  40.997840  29.027981   

   horizontal_accuracy  altitude  altitude_accuracy location_method  \
0                 11.9      70.1                0.0           fused   
1                 12.0      58.4                0.0             gps   
2                 13.5     188.3                0.0           fused   
3                 15.9     160.2                5.0           fused   
4                  7.3      12.6                0.0       

In [ ]:
dict_gdfs_polygons = {}
if os.path.exists(polygons_path) and os.path.isdir(polygons_path):
    geojson_files = [f for f in os.listdir(polygons_path) if f.endswith('.geojson')]
    print(f"\n'{polygons_path}' klasöründe bulunan GeoJSON dosyaları: {geojson_files}")

    for file_name in geojson_files:
        polygon_type = file_name.replace('.geojson', '').replace('merged_polygons_', '') # Alan türü için bir isim türetme
        file_path = os.path.join(polygons_path, file_name)
        try:
            gdf_polygon = geopandas.read_file(file_path)
            # CRS kontrolü ve dönüşümü (hedef EPSG:4326)
            if gdf_polygon.crs is None:
                print(f"Uyarı: '{file_name}' için CRS tanımlı değil. EPSG:4326 varsayılıyor.")
                gdf_polygon = gdf_polygon.set_crs("EPSG:4326", allow_override=True)
            elif gdf_polygon.crs.to_string() != "EPSG:4326": # CRS'leri string olarak karşılaştır
                print(f"'{file_name}' CRS'i {gdf_polygon.crs} 'den EPSG:4326'ya dönüştürülüyor.")
                gdf_polygon = gdf_polygon.to_crs("EPSG:4326")

            dict_gdfs_polygons[polygon_type] = gdf_polygon
            print(f"'{file_name}' ({polygon_type}) yüklendi. Boyut: {gdf_polygon.shape}, CRS: {gdf_polygon.crs}")
        except Exception as e:
            print(f"Hata: '{file_name}' yüklenemedi. {e}")

    print(f"\nYüklenen poligon türleri: {list(dict_gdfs_polygons.keys())}")
else:
    print(f"HATA: Poligon klasörü bulunamadı: {polygons_path}")
    print("Lütfen klasör yolunu ve adını kontrol edin.")


'/content/drive/MyDrive/Polygons/' klasöründe bulunan GeoJSON dosyaları: ['airport.geojson', 'lvl2.geojson', 'lvl3.geojson', 'lvl4.geojson', 'poi.geojson', 'lvl1.geojson', 'merged_polygons_clubs.geojson', 'merged_polygons_hotels.geojson', 'merged_polygons_luxury_houses.geojson', 'merged_polygons_turkey_sites.geojson', 'merged_polygons_p_schools.geojson']
'airport.geojson' (airport) yüklendi. Boyut: (108, 9), CRS: EPSG:4326
'lvl2.geojson' (lvl2) yüklendi. Boyut: (81, 26), CRS: EPSG:4326
'lvl3.geojson' (lvl3) yüklendi. Boyut: (974, 26), CRS: EPSG:4326
'lvl4.geojson' (lvl4) yüklendi. Boyut: (12481, 26), CRS: EPSG:4326
'poi.geojson' (poi) yüklendi. Boyut: (196708, 12), CRS: EPSG:4326
'lvl1.geojson' (lvl1) yüklendi. Boyut: (7, 26), CRS: EPSG:4326
'merged_polygons_clubs.geojson' (clubs) yüklendi. Boyut: (31, 3), CRS: EPSG:4326
'merged_polygons_hotels.geojson' (hotels) yüklendi. Boyut: (358, 3), CRS: EPSG:4326
'merged_polygons_luxury_houses.geojson' (luxury_houses) yüklendi. Boyut: (47, 3), 

In [ ]:
# df_mobility DataFrame'inden rastgele 10k satır örnekleme
# Bu hücre, zaman damgası dönüşümleri yapıldıktan SONRA çalıştırılmalıdır.
if 'df_mobility' in locals() and df_mobility is not None:
    original_row_count = df_mobility.shape[0]
    sample_size = 5000000

    if original_row_count > sample_size:
        print(f"\nOrijinal df_mobility boyutu (zaman damgası eklendikten sonra): {original_row_count} satır.")
        df_mobility = df_mobility.sample(n=sample_size, random_state=42) # random_state tekrarlanabilirlik için
        print(f"df_mobility DataFrame'inden rastgele {sample_size} satır örneklendi. Yeni boyut: {df_mobility.shape}")
    else:
        print(f"\ndf_mobility DataFrame'i zaten {sample_size} satırdan az veya eşit ({original_row_count} satır), örnekleme yapılmadı.")
else:
    print("\nUyarı: 'df_mobility' DataFrame'i bulunamadı veya None. Örnekleme adımı atlandı.")

# İstenmeyen sütunları düşür (kullanıcı isteği)\n
print("\n--- Sütun Düşürme Başlıyor (Kullanıcı İsteği) ---")
if 'df_mobility' in locals() and df_mobility is not None:
    columns_to_drop = ['region', 'location', 'method']
    existing_columns_to_drop = [col for col in columns_to_drop if col in df_mobility.columns]
    if existing_columns_to_drop:
        df_mobility = df_mobility.drop(columns=existing_columns_to_drop)
        print(f"Şu sütunlar df_mobility'den düşürüldü: {existing_columns_to_drop}")
        print(f"df_mobility yeni boyutu: {df_mobility.shape}")
    else:
        print(f"Belirtilen sütunlar ({columns_to_drop}) df_mobility'de bulunamadı, düşürme işlemi yapılmadı.")\n
else:
    print("Uyarı: 'df_mobility' DataFrame'i tanımlı değil veya None, sütun düşürme işlemi atlandı.")
print("--- Sütun Düşürme Tamamlandı ---")

In [ ]:
# Zaman Damgası Dönüşümü
if 'df_mobility' in locals() and 'timestamp' in df_mobility.columns:
    try:
        df_mobility['timestamp_dt'] = pd.to_datetime(df_mobility['timestamp'], unit='s')
        # Okunabilir formatta yeni bir sütun (Türkiye saatiyle)
        df_mobility['readable_time'] = df_mobility['timestamp_dt'].dt.tz_localize('UTC').dt.tz_convert('Europe/Istanbul').dt.strftime('%Y-%m-%d %H:%M:%S %Z')
        print("\nZaman damgası dönüştürüldü:")
        print(df_mobility[['timestamp', 'timestamp_dt', 'readable_time']].head())
    except Exception as e:
        print(f"HATA: Zaman damgası dönüştürülürken bir sorun oluştu: {e}")
else:
    print("Uyarı: 'df_mobility' DataFrame'i veya 'timestamp' sütunu bulunamadı. Zaman damgası dönüşümü atlandı.")


Zaman damgası dönüştürüldü:
    timestamp        timestamp_dt            readable_time
0  1716563371 2024-05-24 15:09:31  2024-05-24 18:09:31 +03
1  1716564390 2024-05-24 15:26:30  2024-05-24 18:26:30 +03
2  1716563418 2024-05-24 15:10:18  2024-05-24 18:10:18 +03
3  1716562879 2024-05-24 15:01:19  2024-05-24 18:01:19 +03
4  1716563925 2024-05-24 15:18:45  2024-05-24 18:18:45 +03


In [ ]:
# Mobilite Verisini GeoDataFrame'e Dönüştürme
if 'df_mobility' in locals() and 'longitude' in df_mobility.columns and 'latitude' in df_mobility.columns:
    try:
        gdf_mobility = geopandas.GeoDataFrame(
            df_mobility,
            geometry=geopandas.points_from_xy(df_mobility.longitude, df_mobility.latitude),
            crs="EPSG:4326" # WGS84
        )
        print("\nMobilite verisi GeoDataFrame'e dönüştürüldü.")
        print(gdf_mobility.head())
    except Exception as e:
        print(f"HATA: Mobilite verisi GeoDataFrame'e dönüştürülürken bir sorun oluştu: {e}")
else:
    print("Uyarı: 'df_mobility' DataFrame'i veya 'longitude'/'latitude' sütunları bulunamadı. GeoDataFrame dönüşümü atlandı.")


Mobilite verisi GeoDataFrame'e dönüştürüldü.
           timestamp                            device_aid   latitude  \
11818902  1715381206  579c691c-ad0d-48c1-a537-595019fb8d8f  40.971389   
22210453  1714732842  aac74dea-1d11-62ce-a42a-4719c7913979  40.971727   
2143920   1716228508  c3120561-95b7-4010-9b01-ea7ac2128343  41.019800   
27161578  1714819165  8fd8ac7e-1364-4528-89b5-a1a1f2a55b23  41.010000   
47560420  1716618369  be969d25-6296-68a0-a8c2-54d39012ae5e  40.979356   

          longitude  horizontal_accuracy  altitude  altitude_accuracy  \
11818902  29.061102                 17.1      66.2                0.0   
22210453  29.083531                 10.0      53.6                0.0   
2143920   29.107456                 24.3     166.9                0.0   
27161578  29.160000                  5.7       0.0                0.0   
47560420  29.069597                 15.4     102.6                0.0   

         location_method            ip       os  ...  \
11818902           f

In [ ]:
if 'gdf_mobility' in locals() and gdf_mobility is not None and not gdf_mobility.empty and dict_gdfs_polygons:
    gdf_results = gdf_mobility.copy()
    print(f"DEBUG: gdf_results başlangıç CRS: {gdf_results.crs}, Boyut: {gdf_results.shape}")

    # Mobilite verilerinin bounds'unu al ve biraz genişlet
    mobility_bounds = gdf_results.total_bounds
    buffer_distance = 0.01  # ~1km buffer
    filtered_bounds = [
        mobility_bounds[0] - buffer_distance,  # min_x
        mobility_bounds[1] - buffer_distance,  # min_y
        mobility_bounds[2] + buffer_distance,  # max_x
        mobility_bounds[3] + buffer_distance   # max_y
    ]
    print(f"DEBUG: Mobilite verisi bounds: {mobility_bounds}")
    print(f"DEBUG: Filtreleme için kullanılacak bounds (buffer ile): {filtered_bounds}")

    for polygon_type, gdf_polygon_orig in dict_gdfs_polygons.items():
        print(f"\n--- '{polygon_type}' poligonları ile işleme başlanıyor ---")

        # <<< KULLANICI İSTEĞİ: lvl1 ve lvl2 poligonlarını atla >>>
        if polygon_type in ['lvl1', 'lvl2']:
            print(f"'{polygon_type}' poligon türü kullanıcı isteği üzerine atlanıyor.")
            continue
        # <<< DEĞİŞİKLİK SONU >>>

        if gdf_polygon_orig is None or gdf_polygon_orig.empty:
            print(f"DEBUG: '{polygon_type}' için poligon verisi (gdf_polygon_orig) boş veya None. Bu poligon türü atlanıyor.")
            gdf_results[f'in_{polygon_type}'] = False
            continue

        gdf_polygon = gdf_polygon_orig.copy()
        print(f"DEBUG: gdf_polygon ('{polygon_type}') BAŞLANGIÇ shape: {gdf_polygon.shape}, bounds: {gdf_polygon.total_bounds}")

        # *** YENİ: POLİGONLARI FİLTRELE ***
        # Sadece mobilite verisi alanıyla kesişen poligonları al
        polygon_intersects_mobility = gdf_polygon.bounds.apply(
            lambda row: not (
                row['maxx'] < filtered_bounds[0] or  # Poligon mobilite alanının solunda
                row['minx'] > filtered_bounds[2] or  # Poligon mobilite alanının sağında
                row['maxy'] < filtered_bounds[1] or  # Poligon mobilite alanının altında
                row['miny'] > filtered_bounds[3]     # Poligon mobilite alanının üstünde
            ), axis=1
        )

        gdf_polygon = gdf_polygon[polygon_intersects_mobility]
        print(f"DEBUG: gdf_polygon ('{polygon_type}') FİLTRELENMİŞ shape: {gdf_polygon.shape}, bounds: {gdf_polygon.total_bounds}")

        if gdf_polygon.empty:
            print(f"DEBUG: '{polygon_type}' filtrelemeden sonra boş kaldı. Bu alan için uygun poligon yok.")
            gdf_results[f'in_{polygon_type}'] = False
            continue

        # Geçersiz geometrileri temizle
        if not gdf_polygon.is_valid.all():
            print(f"UYARI: gdf_polygon ('{polygon_type}') geçersiz geometriler içeriyor. Temizlenmeye çalışılıyor...")
            gdf_polygon = gdf_polygon[gdf_polygon.is_valid]
            if gdf_polygon.empty:
                print(f"DEBUG: gdf_polygon ('{polygon_type}') geçersiz geometriler temizlendikten sonra boş kaldı. Atlanıyor.")
                gdf_results[f'in_{polygon_type}'] = False
                continue
            print(f"DEBUG: gdf_polygon ('{polygon_type}') temizlendikten sonra shape: {gdf_polygon.shape}")

        # Poligon bilgi sütunlarını belirle
        polygon_info_columns = [col for col in gdf_polygon.columns if col not in ['geometry', 'index_right']]
        if not polygon_info_columns:
            temp_info_col_name = f'{polygon_type}_type_info'
            gdf_polygon[temp_info_col_name] = polygon_type
            polygon_info_columns = [temp_info_col_name]
            print(f"DEBUG: '{polygon_type}' için özel bilgi sütunu bulunamadı, '{temp_info_col_name}' eklendi.")

        try:
            # CRS kontrolü ve dönüştürme
            if gdf_results.crs != gdf_polygon.crs:
                print(f"UYARI: CRS uyuşmazlığı! gdf_results CRS: {gdf_results.crs}, gdf_polygon ('{polygon_type}') CRS: {gdf_polygon.crs}")
                print(f"'{polygon_type}' gdf_results CRS'ine dönüştürülüyor.")
                gdf_polygon = gdf_polygon.to_crs(gdf_results.crs)

            # DEBUG: Geometri türlerini kontrol et
            print(f"DEBUG: gdf_results geometri türleri: {gdf_results.geometry.geom_type.unique()}")
            print(f"DEBUG: gdf_polygon ('{polygon_type}') geometri türleri: {gdf_polygon.geometry.geom_type.unique()}")

            current_rsuffix = f'_poly_{polygon_type}'
            print(f"DEBUG: sjoin için hazırlanıyor: gdf_results ({gdf_results.shape}), gdf_polygon ('{polygon_type}') ({gdf_polygon.shape})")

            # Mekansal birleştirme
            gdf_joined_temp = geopandas.sjoin(gdf_results,
                                            gdf_polygon[['geometry'] + polygon_info_columns],
                                            how='left',
                                            predicate='intersects',
                                            rsuffix=current_rsuffix)

            print(f"DEBUG: gdf_joined_temp SHAPE for '{polygon_type}': {gdf_joined_temp.shape}")

            # Index sütununu bul
            actual_index_right_col = None
            possible_index_columns = [
                f'index{current_rsuffix}',
                f'index_{current_rsuffix[1:]}',
                'index_right',
                f'index__{polygon_type}',
                f'index__poly_{polygon_type}'
            ]

            for col_name in possible_index_columns:
                if col_name in gdf_joined_temp.columns:
                    actual_index_right_col = col_name
                    print(f"DEBUG: Index sütunu bulundu: '{actual_index_right_col}'")
                    break

            if actual_index_right_col is None:
                index_cols = [col for col in gdf_joined_temp.columns if col.startswith('index')]
                print(f"DEBUG: Bulunan index sütunları: {index_cols}")
                if index_cols:
                    actual_index_right_col = index_cols[0]
                    print(f"DEBUG: İlk index sütunu kullanılıyor: '{actual_index_right_col}'")

            if actual_index_right_col:
                # Eşleşme durumunu kontrol et
                match_series = gdf_joined_temp[actual_index_right_col].notna()
                print(f"DEBUG: '{polygon_type}' için toplam eşleşme sayısı: {match_series.sum()}")
                print(f"DEBUG: '{polygon_type}' için eşleşme oranı: {match_series.sum() / len(gdf_results) * 100:.1f}%")

                # Duplicate index durumunu handle et
                if gdf_joined_temp.index.duplicated().any():
                    print(f"DEBUG: '{polygon_type}' için duplicate index'ler mevcut, gruplandırılıyor...")
                    in_polygon_series = gdf_joined_temp.groupby(gdf_joined_temp.index)[actual_index_right_col].apply(lambda x: x.notna().any())
                else:
                    in_polygon_series = match_series

                # Sonucu ana DataFrame'e ekle
                gdf_results[f'in_{polygon_type}'] = False
                gdf_results.loc[in_polygon_series.index, f'in_{polygon_type}'] = in_polygon_series

                # Poligon bilgilerini aktar
                for col in polygon_info_columns:
                    new_col_name_in_results = f'{polygon_type}_{col}'
                    gdf_results[new_col_name_in_results] = pd.NA

                    source_col_in_joined = None
                    if f'{col}{current_rsuffix}' in gdf_joined_temp.columns:
                        source_col_in_joined = f'{col}{current_rsuffix}'
                    elif col in gdf_joined_temp.columns:
                        source_col_in_joined = col

                    if source_col_in_joined and not gdf_joined_temp.empty:
                        matched_rows = gdf_joined_temp[gdf_joined_temp[actual_index_right_col].notna()]
                        if not matched_rows.empty:
                            if matched_rows.index.duplicated().any():
                                first_match_attributes = matched_rows.groupby(matched_rows.index)[source_col_in_joined].first()
                            else:
                                first_match_attributes = matched_rows[source_col_in_joined]
                            gdf_results.loc[first_match_attributes.index, new_col_name_in_results] = first_match_attributes

                print(f"'{polygon_type}' için birleştirme tamamlandı. Eşleşen nokta sayısı: {gdf_results[f'in_{polygon_type}'].sum()}")
                print(f"DEBUG: '{polygon_type}' True/False dağılımı:")
                print(gdf_results[f'in_{polygon_type}'].value_counts())

            else:
                print(f"HATA: '{polygon_type}' için sjoin sonrası hiçbir index sütunu bulunamadı!")
                gdf_results[f'in_{polygon_type}'] = False

            del gdf_joined_temp
            import gc
            gc.collect()

        except Exception as e:
            print(f"HATA: '{polygon_type}' ile mekansal birleştirme sırasında bir sorun oluştu: {e}")
            import traceback
            traceback.print_exc()
            if f'in_{polygon_type}' not in gdf_results.columns:
                gdf_results[f'in_{polygon_type}'] = False

    print("\nTüm poligonlarla mekansal birleştirme tamamlandı.")
    print("Özet İstatistikler:")
    for p_type in dict_gdfs_polygons.keys():
        if f'in_{p_type}' in gdf_results.columns:
            true_count = gdf_results[f'in_{p_type}'].sum()
            total_count = len(gdf_results)
            percentage = (true_count / total_count) * 100 if total_count > 0 else 0
            print(f"  {p_type}: {true_count}/{total_count} (%{percentage:.1f})")

else:
    print("Uyarı: 'gdf_mobility' (boş, None veya yüklenememiş olabilir) veya 'dict_gdfs_polygons' (boş olabilir) nedeniyle mekansal birleştirme atlandı.")

DEBUG: gdf_results başlangıç CRS: EPSG:4326, Boyut: (1000000, 22)
DEBUG: Mobilite verisi bounds: [29.006357  40.9515875 29.203419  41.07727  ]
DEBUG: Filtreleme için kullanılacak bounds (buffer ile): [np.float64(28.996357), np.float64(40.941587500000004), np.float64(29.213419000000002), np.float64(41.08727)]

--- 'airport' poligonları ile işleme başlanıyor ---
DEBUG: gdf_polygon ('airport') BAŞLANGIÇ shape: (108, 9), bounds: [25.8794965 36.2964189 44.2620814 42.0253638]
DEBUG: gdf_polygon ('airport') FİLTRELENMİŞ shape: (1, 9), bounds: [29.2034381 40.9847639 29.2253223 41.0014284]
DEBUG: gdf_results geometri türleri: ['Point']
DEBUG: gdf_polygon ('airport') geometri türleri: ['MultiPolygon']
DEBUG: sjoin için hazırlanıyor: gdf_results ((1000000, 22)), gdf_polygon ('airport') ((1, 9))
DEBUG: gdf_joined_temp SHAPE for 'airport': (1000000, 31)
DEBUG: Index sütunu bulundu: 'index__poly_airport'
DEBUG: 'airport' için toplam eşleşme sayısı: 0
DEBUG: 'airport' için eşleşme oranı: 0.0%
'airpor

/usr/local/lib/python3.11/dist-packages/geopandas/geodataframe.py:1819: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/usr/local/lib/python3.11/dist-packages/geopandas/geodataframe.py:1819: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/usr/local/lib/python3.11/dist-packages/geopandas/geodataframe.py:1819: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joinin

'poi' için birleştirme tamamlandı. Eşleşen nokta sayısı: 60949
DEBUG: 'poi' True/False dağılımı:
in_poi
False    939051
True      60949
Name: count, dtype: int64

--- 'lvl1' poligonları ile işleme başlanıyor ---
DEBUG: gdf_polygon ('lvl1') BAŞLANGIÇ shape: (7, 26), bounds: [25.5656305 35.8058974 44.8176638 42.297    ]
DEBUG: gdf_polygon ('lvl1') FİLTRELENMİŞ shape: (1, 26), bounds: [25.5656305 39.0719739 31.0126662 42.1048008]
DEBUG: gdf_results geometri türleri: ['Point']
DEBUG: gdf_polygon ('lvl1') geometri türleri: ['MultiPolygon']
DEBUG: sjoin için hazırlanıyor: gdf_results ((1000000, 121)), gdf_polygon ('lvl1') ((1, 26))


/usr/local/lib/python3.11/dist-packages/geopandas/tools/sjoin.py:274: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_reset.reset_index(inplace=True)


DEBUG: gdf_joined_temp SHAPE for 'lvl1': (1000000, 147)
DEBUG: Index sütunu bulundu: 'index__poly_lvl1'
DEBUG: 'lvl1' için toplam eşleşme sayısı: 1000000
DEBUG: 'lvl1' için eşleşme oranı: 100.0%


...

DEBUG: gdf_results geometri türleri: ['Point']
DEBUG: gdf_polygon ('p_schools') geometri türleri: ['Polygon']
DEBUG: sjoin için hazırlanıyor: gdf_results ((1000000, 159)), gdf_polygon ('p_schools') ((157, 3))


/usr/local/lib/python3.11/dist-packages/geopandas/tools/sjoin.py:274: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_reset.reset_index(inplace=True)


DEBUG: gdf_joined_temp SHAPE for 'p_schools': (1001264, 162)
DEBUG: Index sütunu bulundu: 'index__poly_p_schools'
DEBUG: 'p_schools' için toplam eşleşme sayısı: 1788
DEBUG: 'p_schools' için eşleşme oranı: 0.2%
DEBUG: 'p_schools' için duplicate index'ler mevcut, gruplandırılıyor...


/usr/local/lib/python3.11/dist-packages/geopandas/geodataframe.py:1819: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/usr/local/lib/python3.11/dist-packages/geopandas/geodataframe.py:1819: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/usr/local/lib/python3.11/dist-packages/geopandas/geodataframe.py:1819: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joinin

'p_schools' için birleştirme tamamlandı. Eşleşen nokta sayısı: 524
DEBUG: 'p_schools' True/False dağılımı:
in_p_schools
False    999476
True        524
Name: count, dtype: int64

Tüm poligonlarla mekansal birleştirme tamamlandı.
Özet İstatistikler:
  airport: 0/1000000 (%0.0)
  lvl2: 1000000/1000000 (%100.0)
  lvl3: 1000000/1000000 (%100.0)
  lvl4: 1000000/1000000 (%100.0)
  poi: 60949/1000000 (%6.1)
  lvl1: 1000000/1000000 (%100.0)
  clubs: 0/1000000 (%0.0)
  hotels: 9/1000000 (%0.0)
  luxury_houses: 124/1000000 (%0.0)
  turkey_sites: 1695/1000000 (%0.2)
  p_schools: 524/1000000 (%0.1)


In [ ]:
if 'base_drive_path' not in locals():
    base_drive_path = '.' # Mevcut çalışma dizini

# Sonuçları Kaydetme
if 'gdf_results' in locals() and not gdf_results.empty:
    try:
        output_filename_csv = 'final_mobilityv2_analysis_results.csv'
        output_path_csv = os.path.join(base_drive_path, output_filename_csv)

        # GeoDataFrame'i CSV olarak kaydetme
        # Geometri sütunu WKT (Well-Known Text) formatında kaydedilecektir.
        gdf_results.to_csv(output_path_csv, index=False, encoding='utf-8-sig')
        # encoding='utf-8-sig' Türkçe karakterlerin Excel gibi programlarda doğru görüntülenmesine yardımcı olur.

        print(f"\nİşlenmiş veriler başarıyla CSV olarak kaydedildi: {output_path_csv}")
    except Exception as e:
        print(f"HATA: Sonuçlar CSV olarak kaydedilirken bir sorun oluştu: {e}")
elif 'gdf_results' in locals() and gdf_results.empty:
    print("\n'gdf_results' DataFrame'i boş olduğu için sonuçlar kaydedilmedi.")
else:
    print("\n'gdf_results' DataFrame'i bulunamadığı için sonuçlar kaydedilemedi.")



İşlenmiş veriler başarıyla CSV olarak kaydedildi: /content/drive/MyDrive/final_mobilityv2_analysis_results.csv
